In [ ]:
from matplotlib import pyplot as plt
from scipy.optimize import least_squares, fsolve

In [11]:
def eq_cost_discharge_fn(num_cells):
    # Cost of electrochemical cell
    # ignore electronics, piping, valves - just want basic material cap cost?
    # each cell has 2 endplates, 2 current collectors, gde, zinc
    # find cost per cell - do zinc separately
    cell_area = 1 #m2
    cost_an_col = 10 #$/m^2
    cost_cat = 0.1 #$/m^2
    cell_cost = cell_area * (cost_an_col + cost_cat)

    # cost of stack
    stack_cost = num_cells*cell_cost

    # flowrate per cell
    q_cell = 7.96/1000*60 #m^3/hr
    q_air_cell = q_cell*0.00262 #m^3/hr - from PC project

    # hopper
    hopper_cost = 55000

    # costs below this from textbook - in 2010 GC $

    # cost of air blower
    # C = a + bS^n
    blower_a = 4450
    blower_b = 57
    blower_n = 0.8
    blower_s = num_cells*q_air_cell
    blower_cost = blower_a + blower_b*blower_s**blower_n

    # cost of ZnO collection chamber
    # just sized it like the smallest scraped crystallizer
    collector_cost = 10000 + 13200*7**0.8

    '''
    #size of storage tanks
    storage_tank_vol = 0.21 #m^3
    num_storage_tank = 3
    '''

    # rinsing tank
    tank_a = 5800
    tank_b = 1600
    tank_n = 0.7
    rinsing_tank_vol = 10 #m^3
    rinsing_tank_cost = tank_a + tank_b*rinsing_tank_vol**tank_n

    # pumps - 2 for KOH
    pump_a = 8000
    pump_b = 240
    pump_n = 0.9

    koh_pump_s = num_cells * q_cell*1000/3600 #l/s
    koh_pump_cost = pump_a + pump_b*koh_pump_s**pump_n

    #just make rinsing pump same size?
    CEPCI2019 = 607.5
    CEPCI2010 = 532.9
    lf = 1.07 #location factor for west coastt - couldn't find one for hawaii

    textbook_costs = blower_cost + collector_cost + rinsing_tank_cost + 3*koh_pump_cost
    textbook_costs_adj = textbook_costs*CEPCI2019/CEPCI2010*lf

    eq_cost = stack_cost + hopper_cost + textbook_costs_adj
    return eq_cost

def ISBL_fn(eq_cost):
    C = 3.2
    return eq_cost * C

def OSBL_fn(ISBL):
    return ISBL*0.4

def D_and_E_fn(OSBL):
    return OSBL*0.25

def X_fn(OSBL):
    return OSBL*0.1

def fix_cap_fn(num_cells):
    eq_cost = eq_cost_discharge_fn(num_cells)
    print('Equipment cost: ', eq_cost)
    ISBL = ISBL_fn(eq_cost)
    print('ISBL: ', ISBL)
    OSBL = OSBL_fn(ISBL)
    print('OSBL: ', OSBL)
    D_and_E = D_and_E_fn(OSBL+ISBL)
    print('D_and_E: ', D_and_E)
    X = X_fn(OSBL+ISBL)
    print('Contingency: ', X)
    fix_cap = OSBL +ISBL + D_and_E + X
    print('Fixed Cap Cost: ', fix_cap)
    return fix_cap


In [12]:
power_density = 4350 #W/m^2
power = 30 #MW
power = power*1000000 #W
num_cells = power/power_density
num_cells = 6900
fix_cap = fix_cap_fn(num_cells)
print('Num_cells: ',num_cells)
#print(fix_cap)

# zinc and koh costs
# power for a week
week_energy = power*168 #Wh, 168 hours in a week
unit_price_zn = 0.0036 # $/Wh
cost_zn_week = week_energy * unit_price_zn
cost_zn = 3 * cost_zn_week #enough zinc for 3 weeks at max power?
print('Starting Materials')
print('Zinc for 3 weeks: ', cost_zn)

# koh cost
num_cells = 6900
vol_cell = .1 #m^3
vol_cells = vol_cell*num_cells
total_vol = vol_cells * 1.1

unit_price_koh = 270 #$/m^3
cost_koh = unit_price_koh * total_vol
print('KOH: ', cost_koh)

total = fix_cap + cost_zn + cost_koh
print('Total cap cost:', total)



q_cell = 7.96/1000*60 #m^3/hr
q_air_cell = q_cell*0.00262 #m^3/hr - from PC project
blower_s = num_cells*q_air_cell
print('Blower size (m^3/hr): ', blower_s)

koh_pump_s = num_cells * q_cell*1000/3600 #l/s
print('Pump size (L/s): ', koh_pump_s)

Equipment cost:  671715.3103073859
ISBL:  2149488.992983635
OSBL:  859795.597193454
D_and_E:  752321.1475442722
Contingency:  300928.4590177089
Fixed Cap Cost:  4062534.19673907
Num_cells:  6900
Starting Materials
Zinc for 3 weeks:  54432000.0
KOH:  204930.00000000003
Total cap cost: 58699464.19673907
Blower size (m^3/hr):  8.634052800000001
Pump size (L/s):  915.4


In [ ]:
num_cells_array = range(0,5000)
fix_cap_array = []

for n in num_cells_array:
    fix_cap = fix_cap_fn(n)
    fix_cap_array.append(fix_cap/1000000)
    
plt.plot(num_cells_array, fix_cap_array)

In [ ]:
def eq(N, *data):
    num_cells, dummy = N
    cost = data
    return [fix_cap_fn(num_cells) - cost, dummy]

guess = [1000.0, 0]
data = [30000000.0]
sol = fsolve(eq, guess, args=data)
sol_num_cells = sol[0]
sol_fix_cap = fix_cap_fn(sol_num_cells)
print(sol_num_cells)
print(sol_fix_cap)

#num cells is same as active area
area = sol_num_cells
power_density = 4350 #W/m^2
power = area * power_density
print(power)

In [ ]:
power_density = 4350 #W/m^2
power = 30 #MW
power = power*1000000 #W
num_cells = power/power_density
fix_cap = fix_cap_fn(num_cells)
print(num_cells)
print(fix_cap)

In [ ]:
def stack_cost_fn(num_cells):
    cell_area = 1 #m2
    cost_an_col = 10 #$/m^2
    cost_cat = 0.1 #$/m^2
    cell_cost = cell_area * (cost_an_col + cost_cat)

    # cost of stack
    stack_cost = num_cells*cell_cost
    return stack_cost

print(stack_cost_fn(num_cells))

In [ ]:
power_density = 4350 #W/m^2
power = 30 #MW
power = power*1000000 #W
num_cells = power/power_density
num_cells = 6900
fix_cap = fix_cap_fn(num_cells)
print('Num_cells: ',num_cells)
#print(fix_cap)

# zinc and koh costs
# power for a week
week_energy = power*168 #Wh, 168 hours in a week
unit_price_zn = 0.0036 # $/Wh
cost_zn_week = week_energy * unit_price_zn
cost_zn = 3 * cost_zn_week #enough zinc for 3 weeks at max power?
print('Starting Materials')
print('Zinc for 3 weeks: ', cost_zn)

# koh cost
num_cells = 6900
vol_cell = .1 #m^3
vol_cells = vol_cell*num_cells
total_vol = vol_cells * 1.1

unit_price_koh = 270 #$/m^3
cost_koh = unit_price_koh * total_vol
print('KOH: ', cost_koh)

total = fix_cap + cost_zn + cost_koh
print(total)



In [ ]:
power_MW_array = range(1,30)
cost_array = []

for power in power_MW_array:
    power_density = 4350 #W/m^2
    power = power*1000000 #W
    num_cells = power/power_density
    fix_cap = fix_cap_fn(num_cells)
    #print(num_cells)
    #print(fix_cap)

    # zinc and koh costs
    # power for a week
    week_energy = power*168 #Wh, 168 hours in a week
    unit_price_zn = 0.0036 # $/Wh
    cost_zn_week = week_energy * unit_price_zn
    cost_zn = 3 * cost_zn_week #enough zinc for 3 weeks at max power?
    #print('Starting Materials')
    #print('Zinc for 3 weeks: ', cost_zn)

    # koh cost
    num_cells = 6897
    vol_cell = .1 #m^3
    vol_cells = vol_cell*num_cells
    total_vol = vol_cells * 1.1

    unit_price_koh = 270 #$/m^3
    cost_koh = unit_price_koh * total_vol
    #print('KOH: ', cost_koh)

    total = fix_cap + cost_zn + cost_koh
    cost_array.append(total/1000000)
    
plt.plot(power_MW_array, cost_array)
plt.xlabel('Power Capacity (MW)')
plt.ylabel('Cost (million $)')

In [ ]:
zn_en_dens = 700  #Wh/kg
power = 5e6
zn_hr = power/zn_en_dens #kg
print(zn_hr)

In [9]:
###Charge it up bb
def eq_cost_charge_fn():
    # reactor cost - Alibaba lmao
    reactor_cost = 20000 #$
    
    # hopper cost - sof's source
    hopper_cost = 55000 #$
    
    # 2 mixers - 5kW power
    mixer_cost = 17000 + 1130*5**1.05
    
    # 2 pumps
    # pumps - 2 for H2SO4
    pump_a = 8000
    pump_b = 240
    pump_n = 0.9
    el_pump_s = 2/60 #L/s
    el_pump_cost = pump_a + pump_b*el_pump_s**pump_n
    
    #just make water pump same - 3 total pumps
    
    #heat ex - double pipe cause it's the simplest
    cooler_cost = 1900 +2500*1**1
    
    # cost of Zn collection chamber
    # just sized it like conveyor - 10 m long .5 m wide
    conveyor_cost = 41000 + 730*10**1
    
     # rinsing tank
    tank_a = 5800
    tank_b = 1600
    tank_n = 0.7
    rinsing_tank_vol = 10 #m^3
    rinsing_tank_cost = tank_a + tank_b*rinsing_tank_vol**tank_n

    #just make rinsing pump same size?
    CEPCI2019 = 607.5
    CEPCI2010 = 532.9
    lf = 1.07 #location factor for west coastt - couldn't find one for hawaii

    textbook_costs = 2*mixer_cost + 3*el_pump_cost + conveyor_cost + cooler_cost + rinsing_tank_cost + 3*el_pump_cost
    textbook_costs_adj = textbook_costs*CEPCI2019/CEPCI2010*lf

    eq_cost = reactor_cost + hopper_cost + textbook_costs_adj
    return eq_cost

def ISBL_fn(eq_cost):
    C = 3.2
    return eq_cost * C

def OSBL_fn(ISBL):
    return ISBL*0.4

def D_and_E_fn(OSBL):
    return OSBL*0.25

def X_fn(OSBL):
    return OSBL*0.1

def fix_cap_fn_charge():
    eq_cost = eq_cost_charge_fn()
    print('Equipment cost: ', eq_cost)
    ISBL = ISBL_fn(eq_cost)
    print('ISBL: ', ISBL)
    OSBL = OSBL_fn(ISBL)
    print('OSBL: ', OSBL)
    D_and_E = D_and_E_fn(OSBL+ISBL)
    print('D_and_E: ', D_and_E)
    X = X_fn(OSBL+ISBL)
    print('Contingency: ', X)
    fix_cap = OSBL +ISBL + D_and_E + X
    print('Fixed Cap Cost: ', fix_cap)
    return fix_cap



In [10]:
fix_cap_charge = fix_cap_fn_charge()
print(fix_cap_charge)

Equipment cost:  271182.5986641969
ISBL:  867784.3157254301
OSBL:  347113.7262901721
D_and_E:  303724.5105039006
Contingency:  121489.80420156024
Fixed Cap Cost:  1640112.3567210634
1640112.3567210634
